<a href="https://colab.research.google.com/github/brenopapa/DE-openflights/blob/main/DeleteEmptyUserGroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Delete Empty User Group

### REQUIREMENTS

In [6]:
%%capture
!pip install --quiet pycarol[dataframe]
!pip install --quiet pyarrow
!pip install --quiet python-dotenv
!pip install --quiet curlify
!pip install --quiet git+https://github.com/jnefoussi/pytechfin.git

In [7]:
import json, os
from google.colab import files
from pycarol import Carol, ApiKeyAuth, PwdAuth, Staging, DataModel
from pycarol import Storage, Connectors
import pandas as pd
import numpy as np
from pycarol.utils.miscellaneous import drop_duplicated_parquet

### FUNÇÕES


In [8]:

def list_groups(login, params):
  return login.call_api(
          "v3/userGroup",
          method="GET",
          content_type="application/x-www-form-urlencoded",
          params=params)

def list_users(login, mdmGroupId):
  query_params_users = {"pageSize": 0}
  body = {"mustList":[{"mdmFilterType":"TYPE_FILTER","mdmValue":"mdmTenantUser"},{"mdmFilterType":"TERM_FILTER","mdmKey":"mdmUserGroupsIds","mdmValue":mdmGroupId}]}
  return login.call_api(
      "v1/users/filterUsers",
      method="POST",
      content_type="application/json",
      params=query_params,
      data=body
  )

def delete_group(mdmGroupId):
    return login.call_api(
          "v3/userGroup/"+mdmGroupId,
          method="DELETE",
          content_type="application/json")

### CREDENCIAIS (INSERIR ARQUIVO VALIDO LOCALMENTE) E CONFIGURAÇÕES


In [14]:
credentialFile = files.upload()
credentialFile = json.loads(credentialFile["carol.json"].decode("utf-8"))

# Exemplo de arquivo válido:
# {"login": "myuser@totvs.com.br", "password": "mypwd", "connectorId": "myconnectorid", "subdomain": "mytenant"}

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.min_rows', 100)

org = 'myorganization'
tenant = 'mytenant'

Saving carol.json to carol (2).json


In [15]:
carol_app = 'carolapp'
credentials = PwdAuth(credentialFile["login"], credentialFile["password"])

login = Carol(tenant, carol_app, auth=credentials, organization=org )
os.environ.update(credentialFile)

### DELETANDO GRUPOS DE USUÁRIOS VAZIOS

In [16]:

keep = []
remove = []
next_query = True
counter = -1
offset = 0
pageSize = 500
query_params = {"offset": offset, "pageSize": pageSize}


# Guardando os grupos sem usuários que serão removidos
while(next_query):
  # Consulta de grupos
  resp = list_groups(login,query_params)
  if (resp['hits'] and len(resp['hits']) > 0 ):
    for group in resp['hits'] :
      # Consulta usuários
      resp_users = list_users(login,group['mdmId'])
      if resp_users['totalHits'] == 0:
        # Grupo sem usuário
        remove.append(group)
      else:
        keep.append(group)
  # Atualização do contador
  if(counter == -1):
    counter = resp['count']
  else:
    counter = counter + resp['count']
  # Atualizando a condição de parada e indice de busca na paginação
  total_hits = resp['totalHits']
  if(total_hits and total_hits > 0):
    if(total_hits == counter):
      next_query = False  
    else:
      offset = counter
      query_params = {"offset": offset, "pageSize": pageSize}
  else:
    next_query = False
 

In [17]:
#Remoção de grupos
for group in remove:
  try:
    result = delete_group(group['mdmId'])
    #  print(result)
    if result['success']:
      print('Grupo '+ group['mdmName'] + ' removido com sucesso.')
  except:
    print('ERRO ao remover grupo '+ group['mdmName'] )
